In [ ]:
#| default_exp transcribe

In [ ]:
#| export

import os
import whisper
import time
from audio_journal.sounds import *
from audio_journal import notion
from audio_journal.notify import send_email
from audio_journal.gpt import generate_quote

In [ ]:
#| export

# Function to transcribe audio
def transcribe_audio(filename):
    # Load the Whisper model
    model = whisper.load_model("small.en")
    result = model.transcribe(filename)
    return result["text"]

def transcribing_process():
    while True:
        try:
            time.sleep(5)
            print("Checking for completed recordings...")
            for file in os.listdir("completed_recordings"):
                if file.endswith(".wav"):
                    audio_file_path = os.path.join("completed_recordings", file)
                    print(f"Transcribing {audio_file_path}")
                    transcription = transcribe_audio(audio_file_path)
                    print("Transcription:", transcription)
                    notion.append_to_notion(transcription)
                    os.remove(audio_file_path)
                    print(f"Transcription uploaded to Notion and {audio_file_path} deleted.")
                    quote = generate_quote(transcription)
                    send_email("Daily journal", quote)
        except Exception as e:
            print(e)
            print(f"Error while transcribing and uploading audio. Trying again in 5 seconds...")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()